In [1]:
import uuid
import requests

In [2]:
import dialogflow_v2beta1 as dialogflow

In [3]:
session_client = dialogflow.SessionsClient.from_service_account_json("coffee-shop-nqllqr-f810158f10fd.json")
project_id = "coffee-shop-nqllqr"

In [4]:
def detect_intent(session_id, text, language_code="en"):
    """Returns the result of detect intent with texts as inputs.

    Using the same `session_id` between requests allows continuation
    of the conversation."""
    
    session = session_client.session_path(project_id, session_id)

    text_input = dialogflow.types.TextInput(
        text=text, language_code=language_code)

    query_input = dialogflow.types.QueryInput(text=text_input)

    response = session_client.detect_intent(
        session=session, query_input=query_input)

    return response

In [5]:
def call_coco(component_id, session_id, **kwargs):
    return requests.post(
        #"http://127.0.0.1:5000/api/exchange/"
        "https://coco-235210.appspot.com/api/exchange/"
        f"{component_id}/{session_id}",
        json=kwargs, headers={"api-key":"master_key"}).json()

In [6]:
session_id = str(uuid.uuid4())
inputs = ["hi", "john smith", "yes", "I want to check my balance"]
for inp in inputs:
    response = detect_intent(session_id, inp).query_result
    print(f"U({response.intent.display_name}): {response.query_text}")
    print(f"B: {response.fulfillment_text}\n")

U(Default Welcome Intent): hi
B: Good day! I have a lot coffee and snacks. What can I get you to drink?

U(Default Fallback Intent): john smith
B: Sorry, I didn't get that.

U(Default Fallback Intent): yes
B: Sorry, can you say that again?

U(order.last): I want to check my balance
B: Will it be for delivery or pick-up?



In [7]:
session_id = str(uuid.uuid4())

In [8]:
call_coco("register_vp3", session_id, user_input="chen@imperson.com")

{'component_done': False,
 'confidence': 0.5,
 'idontknow': False,
 'response': 'I would like to email you important info about your account. I promise not to send spam and you can opt out at any time.  Please type in your email below (ie john@email.com).',
 'updated_context': {}}

In [29]:

def dialogflow_comp(state, inp):
    interpretation = detect_intent(state["session_id"], inp).query_result
    if interpretation.intent.display_name == "Default Welcome Intent" and not "name" in state:
        return [], "namer_comp", False
    return [interpretation.fulfillment_text], None, False

def get_drink_action(first_name=""):
    if first_name:
        return f"Nice to meet you {first_name}, What can I get you to drink?"
    return "Hi, What can I get you to drink?"

def namer_comp(state, inp):
    response = call_coco("namer_vp3", state["session_id"], user_input=inp)
    if response["component_done"]:
        state["name"] = response["updated_context"]
        first_name = response["updated_context"].get("userfirstName")
        return [get_drink_action(first_name)], None, True
    return [response["response"]], None, False

components_index = {
    "dialogflow_comp": dialogflow_comp,
    "namer_comp": namer_comp
}  

def get_response(state, inp):
    responses, new_comp, need_to_pop = components_index[state["control_stack"][-1]](state, inp)
    if need_to_pop:
        state["control_stack"].pop()
    if new_comp:
        state["control_stack"].append(new_comp)
    if len(responses) == 0:
        return get_response(state, inp)
    
    return state, responses
    

In [ ]:
session_id = str(uuid.uuid4())
state = {"control_stack":["dialogflow_comp"], "session_id":session_id}
while(True):
    inp = input()
    state, responses = get_response(state, inp)
    
    for res in responses:
        print(res) 
    
    

 hi


Can you please tell me your name?


 mor buski


Hello mor. Nice to meet you. Is your last name buski? 


 yes


Hi, What can I get you to drink?


In [12]:
print (state)


{'comp': 'dialogflow', 'session_id': '33257f4e-e5fa-4f13-97fa-11e9be9bfb2c', 'name': {'component_done': 'true', 'userlastName': 'buskilla'}}
